In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from mpl_toolkits.mplot3d import Axes3D
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt # plotting
import numpy as np # linear algebra
import os # accessing directory structure
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from PIL import Image, ImageFile
from IPython.display import display
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import glob
import scipy
from sklearn.metrics import mean_absolute_error
from tensorflow.keras import Model
from tensorflow.keras.regularizers import l2
from tensorflow.keras.layers import Conv2D, BatchNormalization, DepthwiseConv2D, MaxPooling2D, AveragePooling2D, Input, Dense, Dropout, Flatten, ReLU, GlobalMaxPooling2D, GlobalAveragePooling2D

In [ ]:
data = np.load('/kaggle/input/crowd-counting/images.npy')
label = np.load('/kaggle/input/crowd-counting/labels.npy')

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(data, label, test_size = 0.1)

In [ ]:
base_model = keras.applications.MobileNet(
    weights='imagenet',  # Load weights pre-trained on ImageNet.
    input_shape=(480, 640, 3),
    include_top=False, pooling = 'max')

In [ ]:
base_model.summary()

In [ ]:
for layer in base_model.layers:
    print(layer.get_config())
    print()

In [ ]:
def create_model():
    input_ = Input(shape = (480, 640, 3))
    X = BatchNormalization()(input_)
    
    X = Conv2D(filters = 32, kernel_size = (7, 7), activation = 'linear', strides = (1,1), kernel_regularizer = l2(1e-3), use_bias = False)(X)
    X = BatchNormalization()(X)
    X = ReLU()(X)
    X = AveragePooling2D()(X)
    X = Dropout(0.1)(X)
    
    X = Conv2D(filters = 64, kernel_size = (5, 5), activation = 'linear', strides = (1,1), kernel_regularizer = l2(1e-3), use_bias = False)(X)
    X = BatchNormalization()(X)
    X = ReLU()(X)
    X = AveragePooling2D()(X)
    X = Dropout(0.1)(X)
    
    X = Conv2D(filters = 128, kernel_size = (3, 3), activation = 'linear', strides = (1,1), kernel_regularizer = l2(1e-3), use_bias = False)(X)
    X = BatchNormalization()(X)
    X = ReLU()(X)
    X = AveragePooling2D()(X)
    X = Dropout(0.1)(X)
    
    X = GlobalMaxPooling2D()(X)

    output = Dense(1, activation = 'relu')(X)
    model = Model(inputs = input_, outputs = output) 
    return model

model = create_model()
opt = tf.keras.optimizers.Adam(lr=0.001, decay = 1e-5, beta_1=0.9, beta_2=0.999, amsgrad=False)
model.compile(optimizer='adam', loss='mse', metrics = [keras.metrics.MeanAbsoluteError()])

In [ ]:
history = model.fit(x = X_train, y = y_train, epochs = 150,  batch_size = 64, verbose = 1,shuffle = True, validation_split = 0.1)

In [ ]:
model.layers[7]

In [ ]:
model.layers[7].weights

In [ ]:
model.save("model_sota.h5")